# Estruturas de Dados do CPython

## Quem eu sou

+ Cauê Baasch de Souza, 20

+ Graduando de Ciência da Computação na UFSC

+ Defensor do Python 3

+ Aprendiz de Rust

## O que são estruturas de dados

_Sean Parent, ["Better Code: Data Structures", CppCon 2015](https://www.youtube.com/watch?v=sWgDk-o-6ZE)_

+ "Uma estrutura de dados é um formato para organização e armazenamento de dados."

+ "É uma estrutura que usa relações de valor, físicas e representacionais para codificar relações semânticas em uma coleção de objetos."

+ "A escolha de codificação pode fazer uma diferença drástica no desempenho das operações."

## Complexidade Assintótica

+ Tempo em função do tamanho da entrada

+ Big-O é um limite superior (pior caso)

+ O(1) < O(log n) < O(n) < O(n²) < O(2ⁿ)

## [Listas]

+ Sequências (`__len__` e `__getitem__`)

+ Ordenadas

+ Mutáveis

### Implementação

```C
typedef struct {
    PyVarObject ob_base;
    PyObject** ob_item;
    Py_ssize_t allocated;
} PyListObject
```

```C
typedef struct {
    PyObject ob_base;
    Py_ssize_t ob_size;
} PyVarObject;
```

```C
typedef struct _object {
    Py_ssize_t ob_refcnt;
    struct _typeobject* ob_type;
} PyObject;
```

### Implementação

#### Array Dinâmico
+ Rust `Vec`
+ C++ `std::vector`
+ Java `ArrayList`
+ Python `list`

#### Fator de redimensionamento
+ Rust `2x`
+ C++ `2x`
+ Java `1.5x`

+ Python `~1.125x`

```C
newsize + (newsize >> 3) + (newsize < 9 ? 3 : 6)
```

### Complexidade das Operações

```python
>>> l = [...]
```

```python
>>> n = len(l)  # O(1)
```

```python
>>> l.append(x)  # O(1) amortizado
```

```python
>>> l.insert(k, x)  # O(n)
```

```python
>>> l.pop()  # O(1)
```

```python
>>> l[k] = x  # O(1)
```

```python
>>> x in l  # O(n)
```

## (Tuplas,)

+ Sequências
+ Ordenadas

+ **Imutáveis**

### Implementação

+ Semelhante à da lista
+ Espaço pré-alocado para tuplas de tamanho `<=20`
+ Complexidades das operações iguais às da lista

## collections.namedtuple()

+ Cria subclasses de `tuple`

```python
>>> from collections import namedtuple
>>> ColorRGB = namedtuple('ColorRGB',
...                       ('red', 'green', 'blue'))
```

```python
>>> magenta = ColorRGB(255, 0, 255)
>>> magenta
ColorRGB(red=255, green=0, blue=255)
```

```python
>>> magenta = ColorRGB('#FF00FF')
TypeError: __new__() missing 2 required positional arguments: 'green' and 'blue'
```

```python
>>> magenta.red
255
>>> magenta[1]
0
>>> r, g, b = magenta
```

### typing.NamedTuple

```python
>>> from typing import NamedTuple
>>> class ColorRGB(NamedTuple):
...     red: int
...     green: int
...     blue: int
...     
...     def __lt__(self, other: 'ColorRGB'):
...         "less than (<)"
...         return sum(self) < sum(other)
```

## collections.deque

+ Double-ended queue (fila de duas pontas)

+ Operações thread-safe nos dois lados

+ Mais rápido que `queue.Queue`

```python
>>> from collections import deque
>>> d = deque([3, 5, 7], maxlen=10)
>>> d.appendleft(2)
>>> d.append(11); d
deque([2, 3, 5, 7, 11])
>>> d.rotate(2); d
deque([7, 11, 2, 3, 5])
```

### Implementação

+ Lista duplamente encadeada

```C
typedef struct BLOCK {
    struct BLOCK* leftlink;
    PyObject* data[BLOCKLEN];
    struct BLOCK* rightlink;
} block;
```

```C
typedef struct {
    PyVarObject ob_base;
    block* leftblock;
    block* rightblock;
    Py_ssize_t leftindex;
    Py_ssize_t rightindex;
    size_t state;
    Py_ssize_t maxlen;
    PyObject* weakreflist;
} dequeobject;
```

### Complexidade das Operações

```python
>>> d = deque(...)
```

```python
>>> d.append(...)  # O(1)
>>> d.appendleft(...)  # O(1)
```

```python
>>> d.pop(...)  # O(1)
>>> d.popleft(...)  # O(1)
```

```python
>>> d.rotate(k)  # O(k)
```

## {Sets}

+ Conjuntos como os da matemática

+ Não garante ordem

+ Garante unicidade

+ Mutáveis (equivalente imutável: `frozenset`)

+ Implementados como hash tables

+ Usam funções de hash para computar a posição de um item na memória

### Funções hash

+ Propriedade: `a == b -> hash(a) == hash(b)`

```python
>>> hash('python floripa')
8745422742975612035
>>> hash(2017)
2017
>>> hash(6.2831853)
652980844317083654
>>> hash((2017, 11, 25))
-3476838564118747572
```

```python
>>> hash([2, 3, 5, 7, 11])
TypeError: unhashable type: 'list'
```

### Complexidade das Operações

```python
>>> s = {...}
```

```python
>>> s.add(x)  # caso médio: O(1); pior caso: O(n)
>>> s.remove(x)  # caso médio: O(1); pior caso: O(n)
```

```python
>>> x in s  # caso médio: O(1); pior caso: O(n)
```

```python
>>> s.union(t)  # O(len(s) + len(t))
>>> s.intersection(t) 
# caso médio: O(min(len(s), len(t)))
# troque min por max se t não é um set
# pior caso: O(len(s) * len(t))
```

```python
>>> s - t  # O(len(s))
```

## {Dicionários}

+ Mapeia chaves a valores

+ Implementação semelhante à dos sets

+ Mutáveis

+ Não garantem ordem

+ [Modern Python Dictionaries: a confluence of a dozen great ideas](https://www.youtube.com/watch?v=npw4s1QTmPg)
    + _Raymond Hettinger, PyCon 2017_

### Complexidade das Operações

```python
>>> d = {...}
>>> d[x]  # caso médio: O(1); pior caso: O(n)
>>> d[x] = y  # caso médio: O(1); pior caso: O(n)
>>> del d[x]  # caso médio: O(1); pior caso: O(n)
```

### collections.defaultdict

+ Dicionário com factory function para valores padrão

```python
>>> from collections import defaultdict
>>> d = defaultdict(list)
>>> d['a'].append(1)
>>> d
```
```
defaultdict(<class 'list'>, {'a': [1]})
```

```python
>>> d = defaultdict(int)
>>> d['a'] += 1
>>> d['b']
0
>>> d
```
```
defaultdict(<class 'int'>, {'a': 1, 'b': 0})
```

## Conclusão

+ Use a estrutura mais adequada semanticamente

+ Tenha em mente a implementação e a complexidade das operações

+ Conheça a biblioteca padrão da linguagem

# Obrigado!
<center> Cauê Baasch de Souza </center>
<center> t.me/cauebs </center>